# Calculating Average Travel Time to Other Destination Categories
- Create a feature class for each destination type (healthcare, higher education, essential services) that contains the point locations, types, and capacity data.
- For each TAZ, find out how many (and ideally which) destinations of each type are within a buffer of the TAZ centroid.
- Flag each TAZ with whether it meets the threshold for the number of destinations within a buffer around the TAZ centroid for the type of destination.
- Use model skims for travel times from TAZ centroid to TAZ centroid for all of the following modes: Drive (SOV as proxy), Transit, Rapid Transit, Bus, Walk.
- Then pick the minimum time to reach a flagged TAZ and weight by TAZ population and use that for calculation of the average travel time for MPO.
- Then flag origin TAZs as whether their minimum distance to a flagged TAZ is within the average threshold for MPO.


Environment: base_py_37_omx_geop


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
import openmatrix as omx

In [47]:
##Import all the CSVs that contain skims 
Walk_skim= omx.open_file('M:/LRTP/LRTP_AvgTT/out/Walk_skim.omx','r')
#Import AM
DAT_Boat_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_Boat_tr_skim.omx','r')
DAT_CommRail_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_CommRail_tr_skim.omx','r')
DAT_LocalBus_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_LocalBus_tr_skim.omx','r')
DAT_RapidTransit_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_RapidTransit_tr_skim2.omx','r')
WAT_Transit_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/WAT_for_All_tr_skim.omx','r')
SOV_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/SOV_skim.omx','r')

#put AM CSVs into Dictionary
skims_AM = {'DAT_Boat':DAT_Boat_skim_AM, 'DAT_CR':DAT_CommRail_skim_AM, 
            'DAT_LB':DAT_LocalBus_skim_AM, 'DAT_RT':DAT_RapidTransit_skim_AM, 
            'WAT_TR':WAT_Transit_skim_AM, 'SOV':SOV_skim_AM, 'Walk': Walk_skim}

#Import MD
DAT_Boat_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_Boat_tr_skim.omx','r')
DAT_CommRail_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_CommRail_tr_skim.omx','r')
DAT_LocalBus_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_LocalBus_tr_skim.omx','r')
DAT_RapidTransit_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_Rapid_Transit_tr_skim.omx','r')
WAT_Transit_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\WAT_for_All_tr_skim.omx','r')
SOV_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\SOV_skim.omx','r')

#put MD CSVs into Dictionary
skims_MD = {'DAT_Boat':DAT_Boat_skim_MD, 'DAT_CR':DAT_CommRail_skim_MD, 
            'DAT_LB':DAT_LocalBus_skim_MD, 'DAT_RT':DAT_RapidTransit_skim_MD, 
            'WAT_TR':WAT_Transit_skim_MD, 'SOV':SOV_skim_MD, 'Walk': Walk_skim}

In [48]:
##Import TAZ tables for Essential Services, Healthcare, and Higher Ed. 
dest_TAZs = pd.read_csv(r'M:/LRTP/LRTP_AvgTT/MPO_TAZ_ES_1m2.csv',header=0, sep=',',
                        usecols=['taz', 'ES_FLAG_ALL','HIED_Count5', 'HLTH_COUNT1m'])
#create table with column for each destination type that flags (0,1) which TAZs pass the destination threshold
dest_TAZs['HIED_FLAG']=[1 if x >=1 else 0 for x in dest_TAZs['HIED_Count5']]
dest_TAZs['HLTH_FLAG']=[1 if x >=1 else 0 for x in dest_TAZs['HLTH_COUNT1m']]

#bring in population data for later
tot_pop = pd.read_csv(r'M:/LRTP/LRTP_AvgTT/TAZ_Pop_CTPS.csv', header=0, sep=',', usecols=['TAZ_ID', 'Tot_Pop'])
dest_TAZs = dest_TAZs.merge(tot_pop, how='left', left_on='taz', right_on ='TAZ_ID')

#dest_TAZs

In [49]:
#make list of total TAZs (this is for row and column names for matrices)
tazs = dest_TAZs['taz'].tolist()
tazs.sort()
#make lists of only destination TAZs (this is for averaging)
ES_list = dest_TAZs[dest_TAZs['ES_FLAG_ALL']==1]['taz'].tolist()
HLTH_list = dest_TAZs[dest_TAZs['HLTH_FLAG']==1]['taz'].tolist()
HIED_list = dest_TAZs[dest_TAZs['HIED_FLAG']==1]['taz'].tolist()
len(HIED_list)

1594

In [50]:
#import trips matrices
AM_trips = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/AfterSC_Final_AM_Tables.omx','r')
MD_trips = omx.open_file('M:/LRTP/LRTP_AvgTT/out/MD/AfterSC_Final_MD_Tables.omx','r')
#turn into dataframes
#AM_trips_DAT = pd.DataFrame((np.array(AM_trips['DAT_Boat'])+np.array(AM_trips['DAT_CR'])+
                             #np.array(AM_trips['DAT_LB']) +np.array(AM_trips['DAT_RT']))[1:1902, 1:1902],
                                   #index=tazs, columns=tazs).replace(-np.inf, np.nan)
AM_trips_WAT = pd.DataFrame(np.array(AM_trips['WAT']))
                            
#MD_trips_DAT = pd.DataFrame((np.array(MD_trips['DAT_Boat'])+np.array(MD_trips['DAT_CR'])+
                             #np.array(MD_trips['DAT_LB']) +np.array(MD_trips['DAT_RT']))[1:1902, 1:1902],
                                   #index=tazs, columns=tazs).replace(-np.inf, np.nan)
MD_trips_WAT = pd.DataFrame(np.array(MD_trips['WAT']))   

#close omx
AM_trips.close()
MD_trips.close()

#make trip total tables for transit so can do weighted average
#AM_trips_AT = AM_trips_DAT + AM_trips_WAT
#MD_trips_AT = MD_trips_DAT + MD_trips_WAT

In [51]:
##Sum the Skims to get Travel Time per Mode by Time of Day (loop through both AM and MD dictionaries)
AM_Tables = {}
MD_Tables = {}

for x in skims_AM.keys():
    #if x != 'SOV' and x != 'Walk' and x != 'WAT_TR':
        #AM_Tables[x] = pd.DataFrame((np.array(skims_AM[x]['Access Drive Time'])+np.array(skims_AM[x]['Access Walk Time'])
                        #+np.array(skims_AM[x]['Dwelling Time'])+np.array(skims_AM[x]['Egress Drive Time'])
                        #+np.array(skims_AM[x]['Egress Walk Time'])+np.array(skims_AM[x]['In-Vehicle Time'])
                        #+np.array(skims_AM[x]['Initial Wait Time'])
                        #+np.array(skims_AM[x]['Transfer Wait Time'])+np.array(skims_AM[x]['Transfer Walk Time']))[1:1902, 1:1902],
                                   #index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'WAT_TR':
        AM_Tables[x] = pd.DataFrame((np.array(skims_AM[x]['Access Walk Time'])
                        +np.array(skims_AM[x]['Dwelling Time'])
                        +np.array(skims_AM[x]['Egress Walk Time'])+np.array(skims_AM[x]['In-Vehicle Time'])
                        +np.array(skims_AM[x]['Initial Wait Time'])
                        +np.array(skims_AM[x]['Transfer Wait Time'])+np.array(skims_AM[x]['Transfer Walk Time']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
        
        #Calculate Walk here so can use to replace null values in WAT_TR
        AM_Tables['Walk'] = pd.DataFrame((np.array(skims_AM['Walk']['WalkTime']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
        AM_Tables['Walk'][AM_Tables['Walk'] > 60] = None
        
        #replace TAZ to TAZ with walk for WAT
        AM_Tables[x] = AM_Tables[x].combine_first(AM_Tables['Walk'])
        
    if x == 'SOV':
        AM_Tables[x] = pd.DataFrame(np.array(skims_AM[x]['CongTime_wTerminalTimes'])[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    #if x == 'Walk':
        #AM_Tables[x] = pd.DataFrame((np.array(skims_AM[x]['WalkTime']))[1:1902, 1:1902],
                                   #index=tazs, columns=tazs).replace(-np.inf, np.nan)
        #AM_Tables[x][AM_Tables[x] > 60] = None
        
for x in skims_MD.keys():
    #if x != 'SOV' and x != 'Walk' and x != 'WAT_TR':
        #MD_Tables[x] = pd.DataFrame((np.array(skims_MD[x]['Access Drive Time'])+np.array(skims_MD[x]['Access Walk Time'])
                        #+np.array(skims_MD[x]['Dwelling Time'])+np.array(skims_MD[x]['Egress Drive Time'])
                        #+np.array(skims_MD[x]['Egress Walk Time'])+np.array(skims_MD[x]['In-Vehicle Time'])
                        #+np.array(skims_MD[x]['Initial Wait Time'])
                        #+np.array(skims_MD[x]['Transfer Wait Time'])+np.array(skims_MD[x]['Transfer Walk Time']))[1:1902, 1:1902],
                                   #index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'WAT_TR':
        MD_Tables[x] = pd.DataFrame((np.array(skims_MD[x]['Access Walk Time'])
                        +np.array(skims_MD[x]['Dwelling Time'])
                        +np.array(skims_MD[x]['Egress Walk Time'])+np.array(skims_MD[x]['In-Vehicle Time'])
                        +np.array(skims_MD[x]['Initial Wait Time'])
                        +np.array(skims_MD[x]['Transfer Wait Time'])+np.array(skims_MD[x]['Transfer Walk Time']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
        #Calculate walk so that can replace for WAT_TR where TAZ to same TAZ
        MD_Tables['Walk'] = pd.DataFrame((np.array(skims_MD['Walk']['WalkTime']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
        MD_Tables['Walk'][MD_Tables['Walk'] > 60] = None
        #replace TAZ to TAZ with walk for WAT
        MD_Tables[x] = MD_Tables[x].combine_first(MD_Tables['Walk'])
    if x == 'SOV':
        MD_Tables[x] = pd.DataFrame(np.array(skims_MD[x]['CongTime_wTerminalTimes'])[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    #if x == 'Walk':
        #MD_Tables[x] = pd.DataFrame((np.array(skims_MD[x]['WalkTime']))[1:1902, 1:1902],
                                   #index=tazs, columns=tazs).replace(-np.inf, np.nan)
        #MD_Tables[x][MD_Tables[x] > 60] = None
    


C:\Users\matkinson\Anaconda3\envs\base_py_37_omx_geop\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in add
C:\Users\matkinson\Anaconda3\envs\base_py_37_omx_geop\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: overflow encountered in add


In [52]:
#close everything! 
Walk_skim.close()

#DAT_Boat_skim_AM.close()
#DAT_CommRail_skim_AM.close()
#DAT_LocalBus_skim_AM.close()
#DAT_RapidTransit_skim_AM.close()
WAT_Transit_skim_AM.close()
SOV_skim_AM.close()

#DAT_Boat_skim_MD.close()
#DAT_CommRail_skim_MD.close()
#DAT_LocalBus_skim_MD.close()
#DAT_RapidTransit_skim_MD.close()
WAT_Transit_skim_MD.close()
SOV_skim_MD.close()

In [53]:
#np.where(AM_Tables['AM_TR'].loc[:,ES_list].apply(lambda x: x.isin(x.nsmallest(3)), axis=1).loc[1] == True) 
#AM_Tables['AM_TR'].loc[:,ES_list].apply(lambda x: x.isin(x.nsmallest(3)), axis=1)

'''Min_Dest_TAZ={}
TravelTime ={}
for index, row in AM_Tables['AM_TR'].loc[:,ES_list].iterrows():
    Min_Dest_TAZ[index] = list(row.sort_values()[:3].index)#index (e.g. TAZ destinations)
    TravelTime[index]=np.nansum(row.sort_values()[:3].values)/3 # values'''

Min_Dest_TAZ={}
TravelTime ={}
for index, row in AM_Tables['AM_TR'].loc[:,ES_list].iterrows(): #for every row get lowest 3
    Min_Dest_TAZ[index] = list(row.sort_values()[:3].index)#index (e.g. TAZ destinations)
    TravelTime[index]=np.nansum(row.sort_values()[:3].values)/3 # values
#turn into dataframe and join to main table
temp = pd.DataFrame({'Min_Dest_TAZ': Min_Dest_TAZ}, {'TravelTime':TravelTime})
#Avgs['ES_'+x] = Avgs['ES_'+x].merge(right = temp, how = 'left')


KeyError: 'AM_TR'

In [60]:
temp


,Min_Dest_TAZ
TravelTime,NaN


In [54]:
#make DAT tables (get min for each cell!)
#AM_Tables['AM_DAT'] = np.fmin(np.fmin(AM_Tables['DAT_Boat'],AM_Tables['DAT_CR']), np.fmin(AM_Tables['DAT_LB'],AM_Tables['DAT_RT']))
#MD_Tables['MD_DAT'] = np.fmin(np.fmin(MD_Tables['DAT_Boat'],MD_Tables['DAT_CR']), np.fmin(MD_Tables['DAT_LB'],MD_Tables['DAT_RT']))

#Create new table in dictionary: All Transit
    #take the average - weight it by # of trips for each type of transit
#AM_Tables['AM_Transit'] = ((AM_Tables['AM_DAT']*AM_trips_DAT)+(AM_Tables['AM_WAT']*AM_trips_WAT))/(AM_trips_AT)
#MD_Tables['MD_Transit'] = ((MD_Tables['MD_DAT']*MD_trips_DAT)+(MD_Tables['MD_WAT']*MD_trips_WAT))/(MD_trips_AT)
    #take the lowest
#AM_Tables['AM_TR'] = np.fmin(AM_Tables['AM_DAT'], AM_Tables['WAT_TR'])
#MD_Tables['MD_TR'] = np.fmin(MD_Tables['MD_DAT'], MD_Tables['WAT_TR'])
#use these if no drive access transit
AM_Tables['AM_TR'] = AM_Tables['WAT_TR']
MD_Tables['MD_TR'] = MD_Tables['WAT_TR']

In [62]:
##Calculate Averages
    #weighted average of all the lowest travel times from each TAZ to closest (timewise) destination TAZ 
        #(ones that pass the destination threshold (flagged)), weighted by population of the origin TAZ.
    #do this for all modes and TOD - should end up with a number each for SOV, All Transit, Rapid Transit, Local Bus, Commuter Rail, Boat, and Walking for AM and MD.

    #want to end up with a table for each mode for each dest type with these columns: Origin TAZ, Dest TAZ, Time, Tot_Pop
    #need to do AM for Healthcare and Essential Services, do MD for HiEd. (three tables per mode)
Avgs = {}
#Transit - ES
#SOV - ES
#Walking - ES
for x in ['AM_TR', 'SOV', 'Walk']:
    Avgs['ES_'+x] = AM_Tables[x].loc[:,ES_list] #restrict columns to just what is an ES destination TAZ
    #do lowest 3 for transit
    if x == 'AM_TR':
        #get the 3 dest TAZs that are the closest
        Min_Dest_TAZ={}
        TravelTime ={}
        for index, row in AM_Tables['AM_TR'].loc[:,ES_list].iterrows(): #for every row get lowest 3
            Min_Dest_TAZ[index] = str(list(row.sort_values()[:3].index))#index (e.g. TAZ destinations)
            TravelTime[index]=np.nansum(row.sort_values()[:3].values)/3 # values
        #turn into dataframe and join to main table
        temp = pd.DataFrame.from_dict(Min_Dest_TAZ, orient='index', columns=['Min_Dest_TAZ']).join(
            other = pd.DataFrame.from_dict(TravelTime, orient='index', columns=['TravelTime']))
        Avgs['ES_'+x] = Avgs['ES_'+x].join(other = temp, how = 'left')
    else:
        Avgs['ES_'+x]['Min_Dest_TAZ'] = Avgs['ES_'+x].idxmin(axis=1) #new column with the column name of the smallest time in each row
        Avgs['ES_'+x]['TravelTime'] = Avgs['ES_'+x].min(1) #new column with the min val in each row (corresponds with dest TAZ above)
    Avgs['ES_'+x] = Avgs['ES_'+x].reset_index() #turn index into origin taz field
    Avgs['ES_'+x] = Avgs['ES_'+x].rename(columns = {'index':'origin taz'})

    Avgs['ES_'+x] = Avgs['ES_'+x][['origin taz', 'Min_Dest_TAZ', 'TravelTime']] #restrict to a manageable table
#Transit - Healthcare
#Walking - Healthcare
#SOV - Healthcare
for x in ['AM_TR', 'SOV', 'Walk']:
    Avgs['HLTH_'+x] = AM_Tables[x].loc[:,HLTH_list] #restrict columns to just what is an HLTH destination TAZ
    Avgs['HLTH_'+x]['Min_Dest_TAZ'] = Avgs['HLTH_'+x].idxmin(axis=1) #new column with the column name of the smallest time in each row
    Avgs['HLTH_'+x]['TravelTime'] = Avgs['HLTH_'+x].min(1) #new column with the min val in each row (corresponds with dest TAZ above)
    Avgs['HLTH_'+x] = Avgs['HLTH_'+x].reset_index() #turn index into origin taz field
    Avgs['HLTH_'+x] = Avgs['HLTH_'+x].rename(columns = {'index':'origin taz'})

    Avgs['HLTH_'+x] = Avgs['HLTH_'+x][['origin taz', 'Min_Dest_TAZ', 'TravelTime']] #restrict to a manageable table

#SOV - HiEd(MD)
#Transit - HiEd (MD)
#Walking - HiEd
for x in ['MD_TR', 'SOV', 'Walk']:
    Avgs['HIED_'+x] = MD_Tables[x].loc[:,HIED_list] #restrict columns to just what is an HiED destination TAZ
    Avgs['HIED_'+x]['Min_Dest_TAZ'] = Avgs['HIED_'+x].idxmin(axis=1) #new column with the column name of the smallest time in each row
    Avgs['HIED_'+x]['TravelTime'] = Avgs['HIED_'+x].min(1) #new column with the min val in each row (corresponds with dest TAZ above)
    Avgs['HIED_'+x] = Avgs['HIED_'+x].reset_index() #turn index into origin taz field
    Avgs['HIED_'+x] = Avgs['HIED_'+x].rename(columns = {'index':'origin taz'})

    Avgs['HIED_'+x] = Avgs['HIED_'+x][['origin taz', 'Min_Dest_TAZ', 'TravelTime']] #restrict to a manageable table


#if want to not restrict to just transit (e.g. if O and D are same TAZ)
#ES_RT.loc[ES_RT['origin taz'].isin(ES_list), 'Min_Dest_TAZ'] = ES_RT['origin taz'] #if taz is already a dest taz, set D taz to O taz
#ES_RT.loc[ES_RT['origin taz'] == ES_RT['Min_Dest_TAZ'], 'TravelTime'] = None #if the taz is a dest taz, set TT to null
#note here: so there will never be a transit value if the dest taz is the same as the origin taz
#will need to replace with Walk or SOV here - depends on DAT or WAT?
Avgs['ES_AM_TR']

,origin taz,Min_Dest_TAZ,TravelTime
0,1,"[1, 202, 36]",8.922270
1,2,"[2, 202, 35]",7.783895
2,3,"[3, 36, 4]",7.627576
3,4,"[4, 3, 202]",9.946056
4,5,"[5, 3, 41]",7.642837
...,...,...,...
1896,2641,"[2641, 2642, 754]",16.048322
1897,2642,"[2642, 2641, 2644]",27.435572
1898,2643,"[2642, 2644, 754]",14.371516
1899,2644,"[2644, 2642, 754]",16.839678


In [11]:
##Calculate Averages PART 2
    #weighted average of all the lowest travel times from each TAZ to closest (timewise) destination TAZ 
        #(ones that pass the destination threshold (flagged)), weighted by population of the origin TAZ.
    #do this for all modes and TOD - should end up with a number each for SOV, All Transit, Rapid Transit, Local Bus, Commuter Rail, Boat, and Walking for AM and MD.
#actually calculate averages
AvgNums = {}
for x in Avgs.keys():
    Avgs[x] = Avgs[x].merge(dest_TAZs, how = 'left', left_on = 'origin taz', right_on = 'taz') #merge to have all data in one table
    Avgs[x]['TT_Pop'] = Avgs[x]['TravelTime']*Avgs[x]['Tot_Pop'] #calculate time * pop (weight)
    avg = np.nansum(Avgs[x]['TT_Pop'])/np.nansum(Avgs[x]['Tot_Pop']) #make the actual avg, excluding nan values
    AvgNums[x+'_Avg'] = avg #add avg to dictionary with id
    Avgs[x]['Avg'] = avg #add a avg field for calc later
AvgNums

{'ES_AM_TR_Avg': 23.055239813307843,
 'ES_SOV_Avg': 5.628888017575303,
 'ES_Walk_Avg': 8.66032791433802,
 'HLTH_AM_TR_Avg': 26.14402159508965,
 'HLTH_SOV_Avg': 6.4588366804420065,
 'HLTH_Walk_Avg': 10.006462587020875,
 'HIED_MD_TR_Avg': 25.4199342019939,
 'HIED_SOV_Avg': 6.466366023051868,
 'HIED_Walk_Avg': 7.946976722620972}

In [12]:
##Make Flag Tables
#if the TAZ has access to a destination TAZ within the MPO average for that mode, flag.
    #How do I make this into a table?  - maybe copy the TAZ to TAZ table and clear its contents, then just go down the column of all the destination TAZs and query the skim table for that mode and TOD.
    #where column name is in list (aka field in the table that flags destination TAZs for each type)
for x in Avgs.keys():
    Avgs[x]['AvgTT_Flag'] = np.where(Avgs[x]['TravelTime'] <= Avgs[x]['Avg'], 1, 0)


In [13]:
Avgs['HLTH_Walk'].max()

origin taz        2645.000000
Min_Dest_TAZ      2642.000000
TravelTime          51.282684
taz               2645.000000
ES_FLAG_ALL          1.000000
HIED_Count5         49.000000
HLTH_COUNT1m        23.000000
HIED_FLAG            1.000000
HLTH_FLAG            1.000000
TAZ_ID            2645.000000
Tot_Pop           5151.000000
TT_Pop          184343.428539
Avg                 10.006463
AvgTT_Flag           1.000000
dtype: float64

In [14]:
for x in Avgs.keys():
    Avgs[x].to_csv('M:\LRTP\LRTP_AvgTT\Avgs_Walk\Avg_'+x+'.csv',sep = ',')

In [ ]:
#DO NOT RUN (works - use for future)

##Calculate Averages
    #weighted average of all the lowest travel times from each TAZ to closest (timewise) destination TAZ 
        #(ones that pass the destination threshold (flagged)), weighted by population of the origin TAZ.
    #do this for all modes and TOD - should end up with a number each for SOV, All Transit, Rapid Transit, Local Bus, Commuter Rail, Boat, and Walking for AM and MD.

    #want to end up with a table for each mode for each dest type with these columns: Origin TAZ, Dest TAZ, Time, Tot_Pop
    #need to do AM for Healthcare and Essential Services, do MD for HiEd. (three tables per mode)
ES_RT = AM_Tables['DAT_RT'].loc[:,ES_list] #restrict columns to just what is an ES destination TAZ
ES_RT['Min_Dest_TAZ'] = ES_RT.idxmin(axis=1) #new column with the column name of the smallest time in each row
ES_RT['TravelTime'] = ES_RT.min(1) #new column with the min val in each row (corresponds with dest TAZ above)
ES_RT = ES_RT.reset_index() #turn index into origin taz field
ES_RT = ES_RT.rename(columns = {'index':'origin taz'})

ES_RT = ES_RT[['origin taz', 'Min_Dest_TAZ', 'TravelTime']] #restrict to a manageable table

#if want to not restrict to just transit (e.g. if O and D are same TAZ)
#ES_RT.loc[ES_RT['origin taz'].isin(ES_list), 'Min_Dest_TAZ'] = ES_RT['origin taz'] #if taz is already a dest taz, set D taz to O taz
#ES_RT.loc[ES_RT['origin taz'] == ES_RT['Min_Dest_TAZ'], 'TravelTime'] = None #if the taz is a dest taz, set TT to null
#note here: so there will never be a transit value if the dest taz is the same as the origin taz
#will need to replace with Walk or SOV here - depends on DAT or WAT?
ES_RT